# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 12)
pd.set_option('display.width', 300)
df = pd.read_csv("ds_salaries.csv")
df.head()
print(df.dtypes)

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [2]:
from sklearn.model_selection import train_test_split

y = df['salary_in_usd']
X = df.drop(columns=['salary_in_usd', 'salary'])
# удаляю salary вместе с целевой, потому что это по факту одна и та же переменная, в зависимости от валюты, если не удалить - это будет единственный важный признак (с валютами, конечно) и наша модель будет бесполезной, потому что переводить валюты мы и в калькуляторе умеем

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1,
                                                              random_state=52)  # full_train - 0.9, test - 0.1
# 0.1 / 0.9 = 1/9 = 0.(1)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.111111111,
                                                  random_state=52)  # train - 0.8, val - 0.1

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

numeric_features = X_train.select_dtypes(include=["int64"]).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_cat = enc.fit_transform(X_train[categorical_features])
X_val_cat = enc.transform(X_val[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_num = X_train[numeric_features].to_numpy()
X_val_num = X_val[numeric_features].to_numpy()
X_test_num = X_test[numeric_features].to_numpy()

X_train_encoded = np.hstack([X_train_cat, X_train_num])
X_val_encoded = np.hstack([X_val_cat, X_val_num])
X_test_encoded = np.hstack([X_test_cat, X_test_num])

linreg = LinearRegression()
linreg.fit(X_train_encoded, y_train)

y_val_predicted = linreg.predict(X_val_encoded)

print('MAPE: ', mean_absolute_percentage_error(y_val, y_val_predicted))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_val_predicted)))

MAPE:  0.3308699666064591
RMSE:  44494.04669529134


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [4]:
from xgboost.sklearn import XGBRegressor

params = {
    'max_depth': [3, 5, 7, 10],  # максимальная глубина дерева
    'learning_rate': [0.01, 0.05, 0.1, 0.3],  # eta
    'n_estimators': [100, 200, 300, 500],  # количество деревьев
    'gamma': [0, 0.1, 0.5, 1.0],  # минимальное снижение функции потерь для разбиения
    'subsample': [0.6, 0.8, 1.0],  # доля выборки для каждого дерева
    'colsample_bytree': [0.6, 0.8, 1.0]  # доля используемых признаков для каждого дерева
}

xgb = XGBRegressor(random_state=52)

In [5]:
from sklearn.model_selection import RandomizedSearchCV
import time

# Для подбора гиперпараметров из такой огромной сетки разумно использовать randsearchcv
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=52,
    n_jobs=-1
)

start_time = time.time()
random_search.fit(X_train_encoded, y_train)
train_time = time.time() - start_time

best_model = random_search.best_estimator_
print("Лучшие гиперпараметры:", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Лучшие гиперпараметры: {'subsample': 1.0, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.5, 'colsample_bytree': 0.8}


In [6]:
start_time = time.time()
y_val_predicted = best_model.predict(X_val_encoded)
predict_time = time.time() - start_time

print('MAPE: ', mean_absolute_percentage_error(y_val, y_val_predicted))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_val_predicted)))
print('Training time:', train_time)
print('Predict time:', predict_time)

MAPE:  0.31321457028388977
RMSE:  42694.73709955362
Training time: 9.571316480636597
Predict time: 0.00211334228515625


Как мы видим, наше качество модели слегка увеличилось, но недостаточно для того, чтобы мы останавливались.

## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [7]:
from catboost import CatBoostRegressor

params = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'iterations': [100, 200, 300, 500],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

catboost = CatBoostRegressor(verbose=0, random_state=52)

In [8]:
random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=52,
    n_jobs=-1
)

start_time = time.time()
random_search.fit(X_train_encoded, y_train)
train_time = time.time() - start_time

best_model = random_search.best_estimator_
print("Лучшие гиперпараметры:", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Лучшие гиперпараметры: {'subsample': 1.0, 'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 300, 'depth': 6}


In [9]:
start_time = time.time()
y_val_predicted = best_model.predict(X_val_encoded)
predict_time = time.time() - start_time

print('MAPE: ', mean_absolute_percentage_error(y_val, y_val_predicted))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_val_predicted)))
print('Training time:', train_time)
print('Predict time:', predict_time)

MAPE:  0.3146968257924399
RMSE:  42801.886596317905
Training time: 39.08307600021362
Predict time: 0.02188420295715332


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [10]:
from catboost import Pool

train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features.tolist())
val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_features.tolist())

params = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'iterations': [100, 200, 300, 500],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

catboost = CatBoostRegressor(verbose=0, random_state=52)

In [11]:
random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=52,
    n_jobs=-1
)

start_time = time.time()
random_search.fit(X_train, y_train,
                  cat_features=categorical_features.tolist())  # используем некодированные датасеты, потому что catboost сам их обрабатывает
train_time = time.time() - start_time

best_model = random_search.best_estimator_
print("Лучшие гиперпараметры:", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Лучшие гиперпараметры: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 7, 'iterations': 500, 'depth': 6}


In [12]:
start_time = time.time()
y_val_predicted = best_model.predict(X_val)
predict_time = time.time() - start_time

print('MAPE: ', mean_absolute_percentage_error(y_val, y_val_predicted))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_val_predicted)))
print('Training time:', train_time)
print('Predict time:', predict_time)

MAPE:  0.3376371170395717
RMSE:  43202.165723559854
Training time: 237.66255903244019
Predict time: 0.0021042823791503906


**Ответ:** На удивление, у меня catboost с pool показал себя сильно хуже, чем обычный catboost с encoded датасетом. Метрики ошибок выросли (MAPE на 2%, RMSE на 500), а время стало аномально большим. Вероятно, время обучения такое огромное из-за большого количества деревьев (100-500) и маленького learning rate. Результат стал хуже, скорее всего, из-за большего смещения на увеличение количества деревьев, отсюда увеличение коэффициента регуляризации для баланса и предотвращения переобучения с таким количеством деревьев и маленький lr, модель стала чуть более сложной - с колебаниями, которые оказались лишние, похоже на небольшое переобучение.

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [15]:
from lightgbm import LGBMRegressor

params = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300, 500],
    'num_leaves': [15, 31, 63, 127],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

lgbm = LGBMRegressor(random_state=52, verbose=-1)

In [16]:
random_search = RandomizedSearchCV(
    estimator=lgbm,
    param_distributions=params,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    random_state=52,
    n_jobs=-1
)

start_time = time.time()
random_search.fit(X_train_encoded, y_train)
train_time = time.time() - start_time

best_model = random_search.best_estimator_
print("Лучшие гиперпараметры:", random_search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Лучшие гиперпараметры: {'subsample': 0.6, 'num_leaves': 15, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}


In [18]:
start_time = time.time()
y_val_predicted = best_model.predict(X_val_encoded)
predict_time = time.time() - start_time

print('MAPE: ', mean_absolute_percentage_error(y_val, y_val_predicted))
print('RMSE: ', np.sqrt(mean_squared_error(y_val, y_val_predicted)))
print('Training time:', train_time)
print('Predict time:', predict_time)

MAPE:  0.3280455211120822
RMSE:  43398.98154472531
Training time: 19.811838388442993
Predict time: 0.0025081634521484375


C:\Users\Dmitriy\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [21]:
results_table = np.array([["Модель", "MAPE", "RMSE", "Training time", "Predict time"],
                          ["LinearRegression", "0.331", "44.494", "-", "-"],
                          ["XGBRegressor", "0.313", "42.695", "9.57", "0.002"],
                          ["CatBoost (encoded)", "0.315", "42.802", "39.08", "0.022"],
                          ["CatBoost (non-encoded, Pool)", "0.338", "43.202", "237.66", "0.002"],
                          ["LGBMRegressor", "0.328", "43.399", "19.81", "0.0025"]])
results_table

array([['Модель', 'MAPE', 'RMSE', 'Training time', 'Predict time'],
       ['LinearRegression', '0.331', '44.494', '-', '-'],
       ['XGBRegressor', '0.313', '42.695', '9.57', '0.002'],
       ['CatBoost (encoded)', '0.315', '42.802', '39.08', '0.022'],
       ['CatBoost (non-encoded, Pool)', '0.338', '43.202', '237.66',
        '0.002'],
       ['LGBMRegressor', '0.328', '43.399', '19.81', '0.0025']],
      dtype='<U28')

**Ответ:** 
В интересах удобства я сделал целую таблицу с результатами. По ней чётко видно следующее: лидер по всем параметрам сразу - XGBoost. Худшие результаты распределены по разным моделям: худший MAPE у Pool CatBoost, RMSE - LinReg, TrainingTime - Pool CatBoost, Predict time - CatBoost. Однако самая удобная оказалась CatBoost Pool модель, потому что она использует свои встроенные алгоритмы кодирования категориальных данных, поэтому умеет работать без предобработки.

Подбор гиперпараметров по всем моделям следующий:
XGboost: {'subsample': 1.0, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.5, 'colsample_bytree': 0.8}
CatBoost: {'subsample': 1.0, 'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 300, 'depth': 6}
CatBoost+Pool: {'subsample': 0.8, 'learning_rate': 0.05, 'l2_leaf_reg': 7, 'iterations': 500, 'depth': 6}
LightGBM: {'subsample': 0.6, 'num_leaves': 15, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}

Заметно, что почти везде количество деревьев - 200-300, но на CatBoost+Pool - 500, что может указывать на переобучение, большую регуляризацию для баланса и маленький lr. Max depth везде 3 или 6, что довольно очевидно (у нас не "пни" и не "секвойи" на 50 ветвлений, обычные небольшие решающие деревья для бустинга). Интересно, что subsample у lgbm довольно маленький - 60% выборки, в остальном параметры уникальные для каждой модели.

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd

ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true",
                        engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [ ]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
# -- YOUR CODE HERE --
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
ben harper,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine

centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --